In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_ollama import OllamaLLM
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# 1. 시각화 설정
%matplotlib inline

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid", font='NanumGothic') # Seaborn 테마 설정

# 2. 분석할 CSV 파일 로드 
file_path = "./input/중도탈락 학생 현황 (대학)_2026-01-14175237927.csv" #
df = pd.read_csv(file_path, encoding='utf-8')  # 'cp949')

print("데이터 로드 완료")
display(df.head()) 
print(len(df))

# 3. 모델 및 에이전트 설정
model_name = "gemma3:27b"  # "gemma3:27b"

llm = OllamaLLM(model=model_name, 
                temperature=0,
                # 모델이 한 번의 응답에서 생성할 최대 토큰 수 (설명 길이를 강제 제한) 길게 설명하는 것을 방지
                num_predict=2048,  # # 512,  # 이상한 소리하면 줄여보기
               )
llm_with_stop = llm.bind(stop=["Observation:", "\nObservation", "Thought:", "\n\tObservation"])  # 중지 시퀀스 설정
# 에이전트가 지켜야 할 철저한 규칙 정의
custom_prefix = """
You are a specialized Python data analysis expert.
Your ONLY goal is to solve the user's request using the dataframe `df`.

You must use the following format:

[REPORT FORMAT 예시]
Final Answer:
==================================================
            분석 결과 보고서
==================================================
1. 분석 요약: (전체적인 분석 결과를 한 줄 요약)

2. 주요 통계 수치:
   (평균값, 분산, 상관분석, 분산분석, 회귀분석, 요인분석, 주성분분석)

3. 도표 (Visual Analysis):
   - 분석 결과 그래프를 'chart.png'로 저장하고 화면에 출력하였습니다.

4. 데이터 상세 분석:
   - (데이터의 흐름과 특징을 구체적인 숫자를 인용하여 설명)

5. 비즈니스 제언:
   - (분석 결과를 바탕으로 서비스 운영에 도움이 될 만한 아이디어)
==================================================
""".format(col_list=list(df.columns))

agent = create_pandas_dataframe_agent(
    llm_with_stop,  # llm,
    df,
    verbose=True,               # Thought/Action/Observation 실시간 출력
    allow_dangerous_code=True,  # 코드 실행 허용
    # 실행 엔진(AgentExecutor)에 전달할 인자를 별도 딕셔너리로 분리 (에러 발생 시 모델에게 던져줄 가이드 문구)
    agent_executor_kwargs={
        "handle_parsing_errors": (
            "Check your format. Ensure 'Action: python_repl_ast' is present "
            "and 'Action Input' contains ONLY raw Python code without any backticks or markdown tags."
            "You MUST use the format: 'Thought: <reasoning>\nAction: python_repl_ast\nAction Input: <code>'. "
            "Do not explain functions. Just execute the code for the given 'df'."
        ),
    },
    agent_type="zero-shot-react-description",
    max_iterations=100, # 파싱 에러 시 재시도 횟수
    prefix=custom_prefix,
)

# 4. 분석 상세 요구 사항 입력 TODO 테스트용
summary_req = "전체적인 통계값 요약해줘"
hypo_req = "학업중단 사유별 학업중단율을 비교하고 가설을 설정하여라"
result_req = "가설을 검증하기 위해 실제 코드를 실행한 통계적 근거를 보여줘"
biz_req = "분석 결과를 바탕으로 한 분석 전략을 제안해줘"

# 5. 쿼리 조합
full_query = f"""
아래 요청사항에 맞춰 데이터프레임을 분석하고 한국어로 보고서를 작성하세요:

1. 데이터 요약: {summary_req}
2. 가설 설정: {hypo_req}
3. 분석 결과: {result_req}
4. 비즈니스 제언: {biz_req}

반드시 파이썬 코드를 실행하여 얻은 통계 수치를 바탕으로 답변하세요.
"""

# 6. 실행
print(f"{model_name} AGENT GO!GO!GO!...\n")
try:
    response = agent.run(full_query)

    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

데이터 로드 완료


,기준연도,학교종류,설립구분,지역,상태,학교,재적학생\n(A),계(B),미등록,미복학,...,계(B`),미등록.1,미복학.1,자퇴.1,학사경고.1,학생활동.1,유급제적.1,재학연한초과.1,기타.1,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100
0,2024,대학교,사립,경남,기존,가야대학교(김해),"2,012",179,14,30,...,42,2,0,40,0,0,0,0,0,11.4
1,2024,대학교,사립,경기,기존,가천대학교,"28,264","1,090",133,117,...,451,14,0,437,0,0,0,0,0,8.6
2,2024,대학교,사립,강원,기존,가톨릭관동대학교,"8,410",627,4,183,...,127,1,0,126,0,0,0,0,0,10.8
3,2024,대학교,사립,충북,기존,가톨릭꽃동네대학교,557,26,0,0,...,18,0,0,14,0,0,0,0,4,15.9
4,2024,대학교,사립,경기,기존,가톨릭대학교,"9,794",384,15,61,...,164,3,0,161,0,0,0,0,0,8.8


246
gemma3:27b AGENT GO!GO!GO!...



> Entering new AgentExecutor chain...


/tmp/ipykernel_240911/3162879297.py:209: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = agent.run(full_query)


Thought: I need to get the descriptive statistics of the dataframe.
Action: python_repl_ast
Action Input: df.describe(include=[np.number])NameError: name 'np' is not definedThought: I need to import numpy to use it in the describe function.
Action: python_repl_ast
Action Input: import numpy as np; df.describe(include=[np.number])         기준연도         미복학        학사경고        학생활동        유급제적      재학연한초과  \
count   246.0  246.000000  246.000000  246.000000  246.000000  246.000000   
mean   2024.0  105.658537    7.504065    0.040650    0.166667    1.199187   
std       0.0  119.074069   12.082623    0.197882    0.917331    6.429282   
min    2024.0    0.000000    0.000000    0.000000    0.000000    0.000000   
25%    2024.0   13.250000    0.000000    0.000000    0.000000    0.000000   
50%    2024.0   74.000000    1.000000    0.000000    0.000000    0.000000   
75%    2024.0  154.500000   10.000000    0.000000    0.000000    0.000000   
max    2024.0  889.000000   65.000000    1.000000   1